## **ETL - user_items**
**En este notebook realizaremos la extracción, transformación y carga (ETL) de `user_items`**

In [1]:
# Importando Librerías
import json             # Módulo de codificador y decodificador JSON
import ast              # Módulo de Árboles de Sintaxis Abstracta
import pandas as pd     # Librería para manipular datasets
import pyarrow as pa    # Útil para trabajar con este formato y realizar operaciones de lectura y escritura de datos
import pyarrow.parquet as pq   # Útil para leer y escribir datos en formato Parquet de manera eficiente
import gzip             # Librería para comprimir y descomprimir datos
import os               # creación de directorios y comprobación de existencia

#### **Extracción y exploración**

In [2]:
contenido= []  #Se crea una lista vacía que nos permita guardar el recorrido del buble for a continuación

for i in gzip.open("dataset/users_items.json.gz"):          #Creamos bucle que va a recorrer el dataset
    contenido.append(ast.literal_eval(i.decode('utf-8')))       #Lee y guarda en la lista "contenido"

items= pd.DataFrame(contenido)                                   #Se crea el dataframe en base a la lista

In [3]:
items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [4]:
#info dataset
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


#### **Transformación**

Desanidamos los datos de la columna `items`

In [5]:
# se desanidan los datos de la columna ['items']
it1 = items.explode(['items'])
it2 = pd.json_normalize(it1['items']).set_index(it1['items'].index)
items_ok = pd.concat([it2, it1], axis=1)

In [6]:
#Se creo un diccionario con claves columna y tipo
# columna - almacena los nombres de las columnas
# tipo - almacena tipos de datos únicos para cada columna
data_type_process = {
    "columna": items_ok.columns.tolist(), 
    "tipo": [items_ok[columna].apply(type).unique() for columna in items_ok.columns]
}

#Se visualizara el nombre de la columna y su tipo de dato
data_type = pd.DataFrame(data_type_process)
data_type

,columna,tipo
0,item_id,"[<class 'str'>, <class 'float'>]"
1,item_name,"[<class 'str'>, <class 'float'>]"
2,playtime_forever,[<class 'float'>]
3,playtime_2weeks,[<class 'float'>]
4,user_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,steam_id,[<class 'str'>]
7,user_url,[<class 'str'>]
8,items,"[<class 'dict'>, <class 'float'>]"


Se eliminan las columnas que no se ven como pertinentes para el análisis que se realizará.

In [7]:
items_ok.drop(columns=['items_count',
                           'steam_id',
                           'items',
                           'user_url',
                           'item_name',
                           'playtime_2weeks'], axis=1, inplace=True)

In [8]:
# se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)
items_ok = items_ok[items_ok['playtime_forever'] != 0]

In [9]:
#Convertimos a tipo de datos int
items_ok['playtime_forever'] = items_ok['playtime_forever'].fillna(0).astype('int64')

In [10]:
# conservar las 3 columnas mas importantes para el analisis
items_ok = items_ok[['user_id','item_id','playtime_forever']]

Reseteo el indice de las filas

In [11]:
items_ok.reset_index(drop=True, inplace=True)

Se eliminan los datos nulos por columna

In [12]:
items_ok.dropna(subset=['item_id'])
items_ok.dropna(subset=['playtime_forever'])

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,30,7
2,76561197970982479,300,4733
3,76561197970982479,240,1853
4,76561197970982479,3830,333
...,...,...,...
3302047,76561198329548331,227940,43
3302048,76561198329548331,388490,3
3302049,76561198329548331,521570,4
3302050,76561198329548331,519140,3


Se procede a eliminar los duplicados y valores nulos en caso de haber

In [13]:
items_ok = items_ok.drop_duplicates()
items_ok = items_ok.dropna()

**Almaceno el DataFrame en un formato de archivo Parquet con el fin de mejorar su eficiencia y facilitar su uso en consultas posteriores**

Se crea una copia independiente del DataFrame items_ok y asigna esta copia a la variable items_copy. Esto se hace para evitar que cualquier modificación realizada en la copia afecte al DataFrame original, permitiendo trabajar con los datos de manera segura sin alterar el DataFrame original.

In [14]:
# Generamos copia
items_copy = items_ok.copy()

#### **Guardo incialmente el DF en formato CSV, llamado `user_items.csv`**

In [15]:
ruta_carpeta = "data"

if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

ruta_guardar_csv = 'data/user_items.csv'
items_copy.to_csv(ruta_guardar_csv, index=False, encoding='utf-8')

#### **Convierto el CSV y guardo como user_items.parquet**

In [16]:
#Se lee el archivo csv
items_copy= pd.read_csv("data/user_items.csv") 

#Asigno la ruta donde quiero guardar el parquet con el nombre que va tener
ruta_guardar_parquet= "data/user_items.parquet"

#Transformo a una tabla el archivo csv en parquet
table = pa.Table.from_pandas(items_copy)
pq.write_table(table,ruta_guardar_parquet)